### Домашнее задание №3

1. Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).  
2. На данных из урока изучите влияние гиперпараметров на ошибку алгоритма. Подберите аргументы функции `eval_LR_model` для логистической регрессии таким образом, чтобы log loss не превышал значение `0.3` `(0.1).` Как изменились веса?
3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса "1". На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`.
4. Создайте функцию `calc_pred`, возвращающую предсказанные классы (0 или 1). На вход функции подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, а также порог вероятности.
5. (\*) Напишите функции для расчета accuracy, матрицы ошибок, precision и recall, а также F1-score.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
plt.style.use('seaborn-ticks')
plt.rcParams.update({'font.size': 14})

In [3]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

1. Изменим функцию calc_logloss, добавив в нее eta - минимальное слагаемое, не позволяющее выражению под знаком логарифма обращаться в ноль

In [19]:
def calc_logloss(y, y_pred):
    eta = 1e-10
    err = np.mean(- y * np.log(y_pred + eta) - (1.0 - y) * np.log(1.0 - y_pred + eta))
    return err

In [5]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика
X.shape

(10, 4)

In [14]:
def standardization(x: np.ndarray) -> np.ndarray:
    s = (x - x.mean()) / x.std()
    return s

X_st = X.copy()
for dim in range(1, X.shape[1]):
    X_st[:, dim] = standardization(X[:, dim])
    print(f'dim {dim} mean = {X_st[:, dim].mean():.5}, std = {X_st[:, dim].std():.5}')

dim 1 mean = 3.3307e-17, std = 1.0
dim 2 mean = -4.1633e-17, std = 1.0
dim 3 mean = 8.3267e-17, std = 1.0


In [15]:
# логистическая регрессия
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot((y_pred - y), X))
        if i % (iterations / 10) == 0:
            print(i, w, err)
    return w

In [23]:
w = eval_LR_model(X_st, y, 1000, 1e-1)

100 [ 0.09138926 -1.16094315 -0.1252748   1.70944249] 0.46177101829079215
200 [ 0.06253061 -1.50675851 -0.29823305  2.25825345] 0.41604957215320104
300 [ 0.09256514 -1.77156832 -0.44362579  2.70204923] 0.3869501214705787
400 [ 0.13199649 -1.98382095 -0.57262313  3.07426168] 0.36667479541944453
500 [ 0.17350868 -2.15920337 -0.69025032  3.39578815] 0.35164994766055085
600 [ 0.21534862 -2.30767355 -0.79945162  3.68001191] 0.33996597531378125
700 [ 0.25687057 -2.43591323 -0.90209026  3.93590336] 0.33052558576898916
800 [ 0.29778247 -2.5485918  -0.99941183  4.16970095] 0.3226603426702138
900 [ 0.33794907 -2.64907692 -1.09228735  4.38588075] 0.31594272874355467
1000 [ 0.37731566 -2.73985947 -1.18135067  4.58774123] 0.31008783802148143


2. Изменим параметры обучения, чтобы получить ошибку меньше 0.1

In [29]:
w = eval_LR_model(X_st, y, 5000, 1)

500 [ 1.53744071 -4.59400567 -3.51919843  9.42674354] 0.22241065776441604
1000 [ 2.48597049 -6.07038967 -5.35080865 13.29834441] 0.17912710321557784
1500 [ 3.17870548 -7.1980987  -6.71032387 16.21962402] 0.15472071662884251
2000 [ 3.72326319 -8.10878236 -7.79840001 18.57323654] 0.13896409138859428
2500 [ 4.17241929 -8.87258999 -8.71130207 20.55192377] 0.12786845154013163
3000 [ 4.55566432 -9.53114771 -9.5030705  22.26681367] 0.11955657622276428
3500 [  4.89094118 -10.11088404 -10.20675683  23.78713854] 0.11303748679490455
4000 [  5.18989347 -10.62950709 -10.84388812  25.15863368] 0.10774131273899668
4500 [  5.46046813 -11.09941358 -11.42918619  26.41294426] 0.10331781430449578
5000 [  5.70831272 -11.52960113 -11.97311664  27.57279891] 0.09954002718949592


Веса классификатора изменились в сторону увеличения для положительных и в сторону уменьшения для отрицательных значений. 

3. Напишем функцию calc_pred_proba

In [30]:
def calc_pred_proba(X, w):
    return sigmoid(np.dot(X, w))

In [31]:
calc_pred_proba(X_st, w)

array([0.33471231, 0.00359229, 1.        , 0.04084936, 0.94899552,
       0.01839952, 1.        , 0.01510054, 0.63359164, 0.99999999])

4. Напишем функцию calc_pred

In [32]:
def calc_pred(X, w, margin = 0.5):
    return np.where(sigmoid(np.dot(X, w)) >=margin, 1, 0)

In [33]:
calc_pred(X_st, w)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])